In [1]:
import csv
import pandas as pd 
import numpy as np

In [4]:
# load the following data : occupation, intresets, work values 
occupation_file= "db_28_0_text/Occupation Data.txt"
intrests_file= "db_28_0_text/Interests.txt"
work_values_file = "db_28_0_text/Work Values.txt"

In [8]:
df_occ = pd.read_csv(occupation_file,delimiter="\t")
df_intrests = pd.read_csv(intrests_file,delimiter="\t")
df_intrests['Element Name']= df_intrests['Element Name'].apply(lambda x: x.lower())
df_intrests= df_intrests.pivot(index="O*NET-SOC Code", columns="Element Name",values="Data Value")

df_work_values = pd.read_csv(work_values_file,delimiter="\t")
df_work_values['Element Name']= df_work_values['Element Name'].apply(lambda x: x.lower())
df_work_values = df_work_values.pivot(index="O*NET-SOC Code", columns="Element Name",values="Data Value")


In [11]:
df_work_values.head()

Element Name,achievement,first work value high-point,independence,recognition,relationships,second work value high-point,support,third work value high-point,working conditions
O*NET-SOC Code,,,,,,,,,
11-1011.00,6.33,3.0,7.00,7.00,5.00,6.0,5.33,1.0,6.33
11-1011.03,6.67,1.0,6.67,6.00,5.00,6.0,3.33,2.0,6.33
11-1021.00,5.33,4.0,6.00,5.67,6.33,6.0,4.67,2.0,6.00
11-1031.00,5.33,4.0,5.00,5.00,5.67,1.0,4.00,3.0,4.33
11-2011.00,5.33,2.0,5.33,5.33,5.00,1.0,4.00,6.0,5.33


In [12]:
data = df_occ.join(df_intrests, on='O*NET-SOC Code',how="inner",rsuffix='_intrests') \
             .join(df_work_values, on='O*NET-SOC Code',how="inner",rsuffix='_work_values')

In [13]:
df_intrests.columns.unique()

Index(['artistic', 'conventional', 'enterprising', 'first interest high-point',
       'investigative', 'realistic', 'second interest high-point', 'social',
       'third interest high-point'],
      dtype='object', name='Element Name')

In [14]:
# print(data.describe(include="all"))
print(data.columns.unique())


Index(['O*NET-SOC Code', 'Title', 'Description', 'artistic', 'conventional',
       'enterprising', 'first interest high-point', 'investigative',
       'realistic', 'second interest high-point', 'social',
       'third interest high-point', 'achievement',
       'first work value high-point', 'independence', 'recognition',
       'relationships', 'second work value high-point', 'support',
       'third work value high-point', 'working conditions'],
      dtype='object')


In [14]:
#save data
data.to_csv("data.csv")

In [15]:

from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder


In [22]:
training_cols= [x for x in list(data.columns.unique()) if x not in ["Title","O*NET-SOC Code","Description"]]
training_cols

['artistic',
 'conventional',
 'enterprising',
 'first interest high-point',
 'investigative',
 'realistic',
 'second interest high-point',
 'social',
 'third interest high-point',
 'achievement',
 'first work value high-point',
 'independence',
 'recognition',
 'relationships',
 'second work value high-point',
 'support',
 'third work value high-point',
 'working conditions']

In [23]:
label_encoder = LabelEncoder()
label_encoder.fit(data["Title"])


LabelEncoder()

In [93]:
x_train, y_train = data[training_cols], data["Title"]
y_train= label_encoder.transform(y_train)
x_train = x_train.rename(columns={'working conditions': 'working_conditions'})
x_train= x_train.reindex(sorted(x_train.columns), axis=1)

In [94]:
x_train

,achievement,artistic,conventional,enterprising,first interest high-point,first work value high-point,independence,investigative,realistic,recognition,relationships,second interest high-point,second work value high-point,social,support,third interest high-point,third work value high-point,working_conditions
0,6.33,2.67,5.33,7.00,5.0,3.0,7.00,2.00,1.33,7.00,5.00,6.0,6.0,3.67,5.33,0.0,1.0,6.33
1,6.67,2.67,4.33,7.00,5.0,1.0,6.67,4.33,1.00,6.00,5.00,6.0,6.0,2.33,3.33,2.0,2.0,6.33
2,5.33,1.00,3.67,7.00,5.0,4.0,6.00,1.33,1.33,5.67,6.33,6.0,6.0,3.33,4.67,4.0,2.0,6.00
3,5.33,3.67,3.00,7.00,5.0,4.0,5.00,3.67,1.00,5.00,5.67,4.0,1.0,4.67,4.00,0.0,3.0,4.33
4,5.33,5.33,4.67,7.00,5.0,2.0,5.33,2.00,1.67,5.33,5.00,3.0,1.0,2.33,4.00,6.0,6.0,5.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991,2.33,1.00,3.00,2.00,1.0,5.0,3.67,1.33,7.00,2.00,3.33,6.0,6.0,1.00,4.67,0.0,2.0,3.33
992,2.33,1.00,4.67,2.00,1.0,5.0,3.67,4.00,7.00,2.33,3.67,6.0,6.0,1.33,5.67,2.0,4.0,3.17
993,2.33,1.00,5.00,1.33,1.0,5.0,3.00,3.67,7.00,2.33,4.33,6.0,4.0,1.00,5.67,2.0,2.0,3.17
994,2.00,1.00,3.67,2.33,1.0,5.0,2.33,1.33,7.00,1.33,4.00,6.0,4.0,1.00,4.33,0.0,2.0,2.83


In [95]:
print(x_train.shape)
print(y_train.shape)

(874, 18)
(874,)


In [96]:

clf = Pipeline(
    steps=[("scaler", StandardScaler()), ("knn", KNeighborsClassifier(leaf_size=1,n_neighbors=11,p=2))]
)

In [97]:
fitted = clf.fit(x_train,y_train)

In [98]:
fitted.score(x_train,y_train)

0.10983981693363844

In [99]:
res= fitted.predict_proba(x_train)

In [100]:
recall_10 = 0
for i in range(len(x_train)):
    top_10= np.argpartition(res[i], -10)[-10:]
    top_10_jobs = label_encoder.inverse_transform(top_10)
    gt = label_encoder.inverse_transform([y_train[i]])
    if gt in top_10_jobs : 
        recall_10 +=1 
        print(gt)

['Chief Executives']
['Chief Sustainability Officers']
['General and Operations Managers']
['Legislators']
['Advertising and Promotions Managers']
['Marketing Managers']
['Sales Managers']
['Administrative Services Managers']
['Financial Managers']
['Treasurers and Controllers']
['Investment Fund Managers']
['Industrial Production Managers']
['Quality Control Systems Managers']
['Biofuels Production Managers']
['Biomass Power Plant Managers']
['Hydroelectric Production Managers']
['Purchasing Managers']
['Transportation, Storage, and Distribution Managers']
['Supply Chain Managers']
['Compensation and Benefits Managers']
['Human Resources Managers']
['Training and Development Managers']
['Farmers, Ranchers, and Other Agricultural Managers']
['Construction Managers']
['Education and Childcare Administrators, Preschool and Daycare']
['Education Administrators, Kindergarten through Secondary']
['Education Administrators, Postsecondary']
['Architectural and Engineering Managers']
['Biofuel

In [101]:
print(recall_10/874)

0.8935926773455377


In [102]:
x_train.iloc[0]

achievement                     6.33
artistic                        2.67
conventional                    5.33
enterprising                    7.00
first interest high-point       5.00
first work value high-point     3.00
independence                    7.00
investigative                   2.00
realistic                       1.33
recognition                     7.00
relationships                   5.00
second interest high-point      6.00
second work value high-point    6.00
social                          3.67
support                         5.33
third interest high-point       0.00
third work value high-point     1.00
working_conditions              6.33
Name: 0, dtype: float64

In [103]:
df_user = pd.read_csv("Data_Sci_Challenge_Datasets/Data_science_challenge_files/User_scores.csv")
#.pivot(index="user_id", columns="element_name",values="score")


In [104]:
df_user

,user_id,element_name,element_group,score,score_max,score_min
0,101,realistic,interests,0.00,40,0
1,101,investigative,interests,5.00,40,0
2,101,artistic,interests,12.50,40,0
3,101,social,interests,16.25,40,0
4,101,enterprising,interests,12.50,40,0
5,101,conventional,interests,3.75,40,0
6,101,achievement,work_values,12.00,30,6
7,101,independence,work_values,12.00,30,6
8,101,recognition,work_values,24.00,30,6
9,101,relationships,work_values,28.00,30,6


In [105]:
arr =['artistic', 'conventional', 'enterprising',
       'investigative', 'realistic',  'social',
       ]

In [106]:
map_intrests = {
    "no high point available": 0,
    "realistic": 1,
    "investigative": 2,
    "artistic": 3,
    "social":4,
    "enterprising":5,
    "conventional":6,
    "no high point available":0,
    "achievement": 1,
    "working Conditions": 2,
    "recognition": 3,
    "relationships": 4,
    "support": 5,
    "independence": 6

}
lst_labels_tops=[ 
       'third interest high-point',
       'second interest high-point', 
       'first interest high-point',
       'third work value high-point',
       'second work value high-point',
       'first work value high-point',
        ]

In [107]:
df_tops =df_user.sort_values('score').groupby(['user_id','element_group'], sort=True).tail(3)
df_tops["tops"] = lst_labels_tops
df_tops["tops_values"] = df_tops["element_name"].map(lambda x : map_intrests[x] )
df_tops 

,user_id,element_name,element_group,score,score_max,score_min,tops,tops_values
2,101,artistic,interests,12.50,40,0,third interest high-point,3
4,101,enterprising,interests,12.50,40,0,second interest high-point,5
3,101,social,interests,16.25,40,0,first interest high-point,4
10,101,support,work_values,20.00,30,6,third work value high-point,5
8,101,recognition,work_values,24.00,30,6,second work value high-point,3
9,101,relationships,work_values,28.00,30,6,first work value high-point,4


In [108]:
df_tops = df_tops.pivot(index="user_id", columns="tops",values="tops_values")
df_user= df_user.pivot(index="user_id", columns="element_name",values="score")


In [109]:
df_user_norm = df_user.join(df_tops,on='user_id',how="inner")

In [110]:
df_user_norm = df_user_norm.reindex(sorted(df_user_norm.columns), axis=1)

In [111]:
df_user_norm

,achievement,artistic,conventional,enterprising,first interest high-point,first work value high-point,independence,investigative,realistic,recognition,relationships,second interest high-point,second work value high-point,social,support,third interest high-point,third work value high-point,working_conditions
user_id,,,,,,,,,,,,,,,,,,
101,12.0,12.5,3.75,12.5,4,4,12.0,5.0,0.0,24.0,28.0,5,3,16.25,20.0,3,5,18.0


In [114]:

res= fitted.predict_proba(df_user_norm)

In [116]:

top_10= np.argpartition(res[0], -10)[-10:]
top_10_jobs = label_encoder.inverse_transform(top_10)
print(top_10_jobs)
# gt = label_encoder.inverse_transform([y_train[i]])


['Critical Care Nurses' 'Medical and Health Services Managers'
 'Family Medicine Physicians' 'Psychiatrists' 'Pediatricians, General'
 'Occupational Therapists'
 'Education Administrators, Kindergarten through Secondary'
 'Chief Executives' 'Judges, Magistrate Judges, and Magistrates'
 'Nurse Practitioners']
